In [1]:
import pandas as pd
import xlrd
import numpy as np

In [2]:
xlsxFP4 = pd.ExcelFile("dataset/cordisfp4projects.xlsx")
xlsxFP5 = pd.ExcelFile("dataset/cordis-fp5projects.xlsx")
xlsxFP6 = pd.ExcelFile("dataset/cordis-fp6projects.xlsx")
xlsxFP7 = pd.ExcelFile("dataset/cordis-fp7projects.xlsx")
xlsxH2020 = pd.ExcelFile("dataset/cordis-h2020projects.xlsx")

In [3]:
dataFP4 = xlsxFP4.parse()
dataFP5 = xlsxFP5.parse()
dataFP6 = xlsxFP6.parse()
dataFP7 = xlsxFP7.parse()
dataH2020 = xlsxH2020.parse()

In [4]:
df4 = dataFP4[['rcn', 'title', 'objective', 'subjects', 'frameworkProgramme']]
df4.frameworkProgramme.replace(to_replace=df4['frameworkProgramme'].astype(str),value='FP4', inplace=True)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:3547: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [5]:
import re

df5 = dataFP5[['rcn', 'title', 'objective', 'subjects', 'frameworkProgramme']]
df5.frameworkProgramme.replace(to_replace=df5['frameworkProgramme'].astype(str),value='FP5', inplace=True)

In [6]:
df6 = dataFP6[['rcn', 'title', 'objective', 'subjects', 'frameworkProgramme']]
df6.frameworkProgramme.replace(to_replace=np.NaN,value='FP6', inplace=True, regex=True)

In [7]:
df7 = dataFP7[['rcn', 'title', 'objective', 'subjects', 'frameworkProgramme']]
df7.frameworkProgramme.replace(to_replace=np.NaN, value='FP7', inplace=True)

In [8]:
df20 = dataH2020[['rcn', 'title', 'objective', 'subjects', 'frameworkProgramme']]
df20.frameworkProgramme.replace(to_replace=np.NaN, value='H2020', inplace=True)

In [9]:
# Compile the 'objective' of all the projects into a list
list_obj = df4['objective'].tolist()
list5 = df5['objective'].tolist()
list6 = df6['objective'].tolist()
list7 = df7['objective'].tolist()
list20 = df20['objective'].tolist()

In [10]:
list_obj.extend(list5)
list_obj.extend(list6)
list_obj.extend(list7)
list_obj.extend(list20)

In [11]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import string

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()


In [12]:
# Remove common words
en_stop.extend(['will', 'develop', 'project', 'research', 'new', 'use', 'european'])

texts_1 = []

for text in list_obj:
    
    try:
        # 'text' is a float type
        text = str(text)
    except UnicodeEncodeError:
        pass
        
    text = text.lower()
    
    # Remove punctuation & numbers
    exclude_punctuation = list(string.punctuation)
    numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
    exclude_punctuation.extend(numbers)
    
    if text != 'nan':
        text = text.replace('%l', '')
        for c in exclude_punctuation:
            text = text.replace(c, " ")
        
        # tokenize document string
        tokens = tokenizer.tokenize(text)
        
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        stopped_tokens = [i for i in stopped_tokens if not i in ['develop', 'understand', 'model', 'eu', 'europe']]
        
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        
        # add tokens to list
        texts_1.append(stemmed_tokens)

In [13]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts_1)

In [14]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts_1]

In [15]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary)

print(ldamodel.print_topics(num_topics=5, num_words=5))

[(0, u'0.014*product + 0.012*energi + 0.012*market + 0.009*technolog + 0.008*develop'), (1, u'0.009*innov + 0.007*research + 0.007*develop + 0.006*train + 0.006*activ'), (2, u'0.010*studi + 0.006*theori + 0.006*chang + 0.006*propos + 0.006*model'), (3, u'0.014*system + 0.009*technolog + 0.008*develop + 0.008*applic + 0.007*base'), (4, u'0.015*cell + 0.007*function + 0.006*diseas + 0.006*studi + 0.006*use')]
